In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:


train = pd.read_csv("/kaggle/input/playground-series-s5e7/train.csv")
test = pd.read_csv("/kaggle/input/playground-series-s5e7/test.csv")

train



In [ ]:
test.shape

In [ ]:


id = test['id']
train.drop(labels='id',axis='columns',inplace=True)
test.drop(labels='id',axis='columns',inplace=True)



In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

In [ ]:
train.dtypes

In [ ]:
train.describe()

In [ ]:


train['Time_spent_Alone'] = train['Time_spent_Alone'].fillna(train['Time_spent_Alone'].mean())
test['Time_spent_Alone'] = test['Time_spent_Alone'].fillna(test['Time_spent_Alone'].mean())

train[train.select_dtypes(include='number').columns] = train.select_dtypes(include='number').fillna(0)
test[test.select_dtypes(include='number').columns] = test.select_dtypes(include='number').fillna(0)

obj_cols = train.select_dtypes(include='object').columns
train[obj_cols] = train[obj_cols].fillna('No')

test[["Stage_fear",'Drained_after_socializing']] = test[["Stage_fear",'Drained_after_socializing']].fillna('No')



In [ ]:


yes_no_cols = ['Stage_fear', 'Drained_after_socializing'] 

for col in yes_no_cols:
    train[col] = train[col].map({'No': 0, 'Yes': 1})
    test[col] = test[col].map({'No': 0, 'Yes': 1})

train['Personality'] = train['Personality'].map({'Extrovert': 0, 'Introvert': 1})

train



In [ ]:
y = train['Personality']
X = train.drop(labels='Personality',axis='columns')

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:


from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import classification_report



In [ ]:


# SVC pipeline (with scaling)
pipeline_svc = Pipeline([
    ('scaler', StandardScaler()),
    ('svc', SVC())
])

# XGB pipeline (without scaling)
pipeline_xgb = Pipeline([
    ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='mlogloss'))
])

# Parameter grids
param_grid_svc = {
    'svc__C': [0.1, 1, 10],
    'svc__kernel': ['linear', 'rbf'],
    'svc__gamma': ['scale', 'auto']
}

param_grid_xgb = {
    'xgb__n_estimators': [50, 100],
    'xgb__max_depth': [3, 5],
    'xgb__learning_rate': [0.05, 0.1]
}

# Grid search
grid_svc = GridSearchCV(pipeline_svc, param_grid_svc, cv=5, scoring='accuracy', n_jobs=-1)
grid_xgb = GridSearchCV(pipeline_xgb, param_grid_xgb, cv=5, scoring='accuracy', n_jobs=-1)

# Fit both
grid_svc.fit(X_train, y_train)
grid_xgb.fit(X_train, y_train)

# Results
print("SVC best params:", grid_svc.best_params_)
print("XGB best params:", grid_xgb.best_params_)

print("\nSVC Test Report:")
print(classification_report(y_test, grid_svc.predict(X_test)))

print("\nXGB Test Report:")
print(classification_report(y_test, grid_xgb.predict(X_test)))



In [ ]:


best_svc_model = grid_svc.best_estimator_
y_pred = pd.Series(best_svc_model.predict(test))

y_pred = y_pred.map({ 0 : 'Extrovert', 1 : 'Introvert'})

df = pd.concat([id, y_pred], axis=1)


df.to_csv('output.csv', index=False)  

